In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
import seaborn as sns
from sklearn.model_selection import cross_val_predict


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
temporal_features_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/temporal_features.csv', index_col=0)
temporal_features_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/temporal_features.csv', index_col=0)

In [ ]:
df_meal = df_meal.rename_axis('s_id')
df_play = df_play.rename_axis('s_id')

all_features_meal = pd.concat([df_meal, temporal_features_meal], axis=1)
all_features_play = pd.concat([df_play, temporal_features_play], axis=1)
all_features_meal

In [ ]:
temporal_features_meal['label'] = df_meal['label']
temporal_features_play['label'] = df_play['label']

In [ ]:
df_meal = temporal_features_meal
df_play = temporal_features_play

In [ ]:
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})

In [ ]:
df_meal

In [ ]:
df_meal = df_meal.rename_axis('s_id')
df_play = df_play.rename_axis('s_id')

In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
df_grouped_meal

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Create a heatmap of the columns in df_meal
df_grouped_meal.corr()

In [ ]:
df_grouped_play.columns

In [ ]:
#df_grouped_meal.drop(columns=['cluster_1', 'cg_low_freq_power', 'child_low_freq_power'], inplace=True)
df_grouped_play.drop(columns=['cluster_1', 'cg_low_freq_power', 'child_low_freq_power'], inplace=True)

In [ ]:
df_grouped_meal.columns

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_meal[[
       'cluster_0', 'cluster_2', 'jump_transitions', 'total_transitions']]
y = df_grouped_meal['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [1, 2],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [1, 2],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [1, 2],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [1, 2],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [1, 2],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [1, 2],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [1, 2],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_play[[
       'cg_autocorrelation_50', 'cg_autocorrelation_100',
       'cg_entropy', 'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power']]
y = df_grouped_play['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4, 6],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4, 6],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2, 4, 6],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4, 6],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4, 6],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4, 6],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [2, 4, 6],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_play[[
       'cg_autocorrelation_50', 'cg_autocorrelation_100',
       'cg_entropy', 'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power']]
y = df_grouped_play['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [4, 6],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [ 4, 6],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [4, 6],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [4, 6],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [4, 6],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [4, 6],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [4, 6],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

## Combined data classification

In [ ]:
combined = pd.concat([df_grouped_meal[['cluster_0', 'cluster_2', 'jump_transitions', 'total_transitions']],
                       df_grouped_play[[ 'cg_autocorrelation_50', 'cg_autocorrelation_100',
       'cg_entropy', 'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power', 'label']]], axis=1)
combined.dropna(inplace=True)

In [ ]:
combined

In [ ]:
X = combined[[
       'cluster_0', 'cluster_2', 'jump_transitions', 'total_transitions',
       'cg_autocorrelation_50', 'cg_autocorrelation_100',
       'cg_entropy', 'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power']]
y = combined['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [4, 6],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [ 4, 6],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [4, 6],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [4, 6],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [4, 6],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [4, 6],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [4, 6],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

# combining speech and temporal play

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
# select only meal group from speech index
speech_meal = speech.xs('meal', level='group')
speech_play = speech.xs('play', level='group')

In [ ]:
#df_grouped_meal = df_grouped_meal.reset_index(level=0, drop=True)
meal_data = pd.concat([df_grouped_meal, speech_meal], axis=1)
meal_data = meal_data.dropna()

In [ ]:
play_data = pd.concat([df_grouped_play, speech_play], axis=1)
#meal_data = meal_data.drop(columns=['index'])
play_data = play_data.dropna()

In [ ]:
# Custom transformer to apply PCA to a subset of features
class SubsetPCA(BaseEstimator, TransformerMixin):
    def __init__(self, subset_features, n_components):
        self.subset_features = subset_features
        self.n_components = n_components
        self.pca = PCA(n_components=n_components)
    
    def fit(self, data, y=None):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        self.pca.fit(data_subset)
        return self
    
    def transform(self, data):
        # Ensure X is a DataFrame
        data_subset = data[self.subset_features]
        data_pca = self.pca.transform(data_subset)
        data_remaining = data.drop(columns=self.subset_features)
        data_pca_df = pd.DataFrame(data_pca, index=data_remaining.index)
        return pd.concat([data_remaining, data_pca_df], axis=1)
    

# Perform a grid search for each classifier
X = play_data[['Proportion speech child',
       'cg_autocorrelation_50', 'cg_autocorrelation_100',
       'cg_entropy', 'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power']]
y = play_data['label'].iloc[:, 0]

subset_pca = ['cg_autocorrelation_50', 'cg_autocorrelation_100',
       'cg_entropy', 'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power']

remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', SubsetPCA(subset_features=subset_pca, n_components=2), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__n_components': [4, 6],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__n_components': [4, 6],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__n_components': [4, 6],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
})]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([ ('preprocessor', preprocessor), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('preprocessor__pca_pipeline__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

# combining all movement features

In [ ]:
df_meal = all_features_meal 
df_play = all_features_play

In [ ]:
df_meal = df_meal.rename_axis('s_id')
df_play = df_play.rename_axis('s_id')

In [ ]:
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})

In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:
df_grouped_meal.columns

In [ ]:


# Perform a grid search for each classifier
X = df_grouped_meal[['cg_movement_mean', 'cg_movement_var',
       'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var',
        'child_movement_min', 'child_movement_max', 'cg_autocorrelation_50', 'cg_autocorrelation_100', 'cg_entropy',
       'cg_slope', 'cg_spectral_centroid', 'cg_signal_power',
         'child_autocorrelation_50',
       'child_autocorrelation_100', 'child_entropy', 'child_slope',
       'child_spectral_centroid', 'child_signal_power']]
y = df_grouped_meal['label']



# Create a GroupKFold object
gkf = KFold(n_splits=5)
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [4,8,10],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [4,8,10],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2,4,6],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [4,8,10],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components':[4,8,10],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [4,8,10],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    }),
    ('mlp', MLPClassifier(max_iter=500), {
        'pca__n_components': [4,8,10],
        'mlp__hidden_layer_sizes': [(3,), (6,), (3, 3)],
        'mlp__activation': ['tanh', 'relu'],
        'mlp__alpha': [0.0001, 0.001, 0.01],
        'mlp__solver': ['adam', 'lbfgs']
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro',  n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro',  n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'accuracy': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),   
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean() 

In [ ]:
svc_rbf	4.5	0.622576	0.600653	0.637504	0.627730	4.2